# Name Deduplicator

## Introduction
Redundancy in formats of names and their parts leads to a lot of confusion while handling data. Having multiple entries for same person leads to increased size of databases and inefficient operations. Thus, deDuplication of entries is an essential task and this script aims to achieve the same

## External Libraries and Pre-requisites
The script runs on Python3 (tested on ver3.6) and requires the following libraries to be installed:
1. nameparser
2. pandas

## Working
The following functions were created:

In [1]:
import pandas as pd
from nameparser.parser import HumanName

### preProcess
This function reads the first name and last name of the entry appends them into a single string and then converts the resulting string into a HumanName object

After that the function reads the Date of Birth of the entry. Since only the last two digits of the year are given in the dob, the function adds them to 1900 if the last two digits are greater than 18 otherwise it adds them to 2000 (assuming people don't tend to live more than 100 years). Then it converts the date to pandas.DateTime objects making it easier to perform date-time operations

In [2]:
def preProcess(names):
    #Converting names to HumanName class objects
    fullName = []
    for x,name in names.iterrows():
        fullName.append(HumanName(name.fn + ' ' + name.ln))
    names['fullName'] = fullName
    
    #Converting dob to pandas.datetime object (assuming that entries are not older than 100 years)
    dobs = []
    for entry in names.dob:
        if(int(entry[-2:]) < 18):
            dob = 2000+int(entry[-2:])
        else:
            dob = 1900+int(entry[-2:])
        dobs.append(str(entry[:-2])+str(dob))
    names.dob = dobs
    names.dob = pd.to_datetime(names.dob)

### generateCross
This function creates a comparison table from the input records. Each record is compared with records following it.

In [3]:
def generateCross(names):
    #Create a new dataframe which compares every entry with every other entry
    entries = []
    for i in range(len(names)):
        for j in range(i+1,len(names)):
            record = {}
            record['fullName1'] = names.fullName[i]
            record['dob1'] = names.dob[i]
            record['gn1'] = names.gn[i]
            record['fullName2'] = names.fullName[j]
            record['dob2'] = names.dob[j]
            record['gn2'] = names.gn[j]
            entries.append(record)
    namesCross = pd.DataFrame(entries)
    count = 0
    for x,name in namesCross.loc[(namesCross.dob1 == namesCross.dob2)].iterrows():
        if(name.fullName1.first == name.fullName1.first):
            count+=1
    print("Number of entries having same date of birth and first name: ", count)
    print("Number of entries having same date of birth and gender: ", len(namesCross.loc[(namesCross.dob1 == namesCross.dob2) & (namesCross.gn1 == namesCross.gn2)]))
    print("Number of entries having same date of birth: ",len(namesCross.loc[(namesCross.dob1 == namesCross.dob2)]))
    return namesCross

### processNames
This function compares two HumanName objects and merges them into single name. This function should only be used when the first name of both the objects is same otherwise there are chances of loss of data. It gives priority to the first object and prefers to assign it's lastname to the resulting object. If the lastname of the first object is empty then it uses the lastname of second object. The remaining names (apart from the first name and last name of the resulting object) are assigned to the middle name. Also single lettered names are checked for being the abbreviation of any of the names assigned.

In [4]:
def processNames(name1, name2):
    names = list((set(name1.middle.split(' ')) | set(name2.middle.split(' ')) | set(name1.last.split(' ')) | set(name2.last.split(' '))) - {''})
    for name in names:
            if(len(name) == 1):
                for othername in names:
                    if(othername[0] == str(name) and othername!=name):
                        names.remove(name)
                        break
    if(name1.last in names):
        names.remove(name1.last)
        name1.middle = ' '.join(names)
    elif(name2.last in names):
        names.remove(name2.last)
        name1.last = name2.last
        name1.middle = ' '.join(names)

### checkSame
Takes the cross record table and compares both entries on each row. If both the entries are of same gender, were born on the same date and have the same first name then there is a high chance that they are the same person. Taking advantage of this, we assume that such entries are actually of the same person and introduce a new attribute 'same' in the cross record table.

In [5]:
def checkSame(names):
    same = []
    for x,entry in names.iterrows():
        if(entry.dob1 == entry.dob2 and entry.gn1 == entry.gn2 and entry.fullName1.first == entry.fullName2.first):
            same.append(1)
        else:
            same.append(0)
    names['same'] = same

### deDuplicate
This function does the actual job. Since each entry is being compared with the following entries in the cross table, the index positions of the first comparison by a new entry can be calculated using the formula for sum of terms in an arithematic progression. Using this, for any given entry, we can find out the entries that have the attribute 'same' as 1. The names of both these entries are combined using processNames function. Since that entry would already have been considered there is no need to re-read it. Thus a list of entries to be deleted is maintained and all entries present in the deleted list are skipped. The function return the resulting dataframe with merged names and only unique entries

In [6]:
def deDuplicate(names):
    namesCross = generateCross(names)
    namesCross.head()
    checkSame(namesCross)
    n = len(names)
    finals = []
    deleted = set()
    for x,name1 in names.iterrows():
        if x in deleted:
            continue
        record = {}
        if(x == 0):
            record['fullName'] = namesCross.fullName1[x]
            record['dob'] = namesCross.dob1[x]
            record['gn'] = namesCross.gn1[x]
            for y,name2 in names.loc[x+1:].iterrows():
                z = y
                z-=1
    #             print(z," ",namesCross.same[z])
                if(namesCross.same[z] == 1):
                    processNames(record['fullName'],namesCross.fullName2[z])
                    deleted.add(y)

        elif(x!=len(names)-1):
            t = sumAP(n-1,x,-1)
    #         print(t)
            record['fullName'] = namesCross.fullName1[t]
            record['dob'] = namesCross.dob1[t]
            record['gn'] = namesCross.gn1[t]
            for y,name2 in names.loc[x+1:].iterrows():
                k = y-x-1
    #             print(t+k)
                if(namesCross.same[t+k] == 1):
#                     print("yeah", x, y, t+k)
                    processNames(record['fullName'],namesCross.fullName2[t+k])
                    deleted.add(y)
        else:
            record['fullName'] = namesCross.fullName1[len(namesCross) - 1]
            record['dob'] = namesCross.dob1[len(namesCross) - 1]
            record['gn'] = namesCross.gn1[len(namesCross) - 1]
        finals.append(record)
#         print(finals)
    finals = pd.DataFrame(finals)
    return finals

### sumAP
Function to calculate the sum of n terms in an Arithematic Progression

In [7]:
def sumAP(a,n,d):
    s = n * (2 * a + (n-1)*d)/2
    return s


In [8]:
names = pd.read_csv("names.csv")

In [9]:
names.head()

,ln,dob,gn,fn
0,SMITH JR,01/03/68,F,WILLIAM
1,ROTHMEYER JR,01/03/68,F,WILLIAM
2,ASBY JR,01/03/68,F,WILLIAM
3,SALTER JR,01/03/68,F,WILLIAM
4,SALTER JR,01/03/68,F,WILLIAM


In [10]:
preProcess(names)

The preProcess function adds the attribute fullName to the dataframe

In [11]:
names.head()

,ln,dob,gn,fn,fullName
0,SMITH JR,1968-01-03,F,WILLIAM,WILLIAM SMITH JR
1,ROTHMEYER JR,1968-01-03,F,WILLIAM,WILLIAM ROTHMEYER JR
2,ASBY JR,1968-01-03,F,WILLIAM,WILLIAM ASBY JR
3,SALTER JR,1968-01-03,F,WILLIAM,WILLIAM SALTER JR
4,SALTER JR,1968-01-03,F,WILLIAM,WILLIAM SALTER JR


The cross record table looks something like this:

In [12]:
namesCross = generateCross(names)
namesCross.head()

Number of entries having same date of birth and first name:  463
Number of entries having same date of birth and gender:  435
Number of entries having same date of birth:  463


,dob1,dob2,fullName1,fullName2,gn1,gn2
0,1968-01-03,1968-01-03,WILLIAM SMITH JR,WILLIAM ROTHMEYER JR,F,F
1,1968-01-03,1968-01-03,WILLIAM SMITH JR,WILLIAM ASBY JR,F,F
2,1968-01-03,1968-01-03,WILLIAM SMITH JR,WILLIAM SALTER JR,F,F
3,1968-01-03,1968-01-03,WILLIAM SMITH JR,WILLIAM SALTER JR,F,F
4,1968-01-03,1962-02-21,WILLIAM SMITH JR,WILLIAM BLAND JR,F,F


As visible in the dataset, 463 out of 463 entries having same date of births have same first names and 435 out of them even have the same gender. Thus it can be taken as a sufficient condition for entries to be regarded as same:

In [13]:
finals = deDuplicate(names)

Number of entries having same date of birth and first name:  463
Number of entries having same date of birth and gender:  435
Number of entries having same date of birth:  463


The deDuplicated data looks like this

In [14]:
finals.head()

,dob,fullName,gn
0,1968-01-03,WILLIAM ASBY SALTER ROTHMEYER SMITH JR,F
1,1962-02-21,WILLIAM SHAFFER BLAND JR,F
2,1962-02-21,BILL BLAND JR,F
3,1954-08-06,WILLIAM BLAND JR,F
4,1953-10-25,WILLIAM SHAFFER BLAND JR,F


The number of entries is reduced to 29 from 103

In [15]:
len(names)

103

In [16]:
len(finals)

39

In [17]:
finals.loc[1:].to_csv("sample_output.csv")